In [ ]:
!pip install unsloth trl peft accelerate bitsandbytes

In [ ]:
!pip install protobuf==3.20.3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.1/162.1 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 5.29.5
    Uninstalling protobuf-5.29.5:
      Successfully uninstalled protobuf-5.29.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ydf 0.13.0 requires protobuf<7.0.0,>=5.29.1, but you have protobuf 3.20.3 which is incompatible.
tensorflow-metadata 1.17.2 requires protobuf>=4.25.2; python_version >= "3.11", but you have protobuf 3.20.3 which is incompatible.
grpcio-status 1.71.2 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.3 which is incompatible.


In [ ]:
from unsloth import FastLanguageModel
import torch

model_name = "unsloth/Phi-3-mini-4k-instruct-bnb-4bit"

max_seq_length = 1024
dtype = None  # Auto detection

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=True,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.8.9: Fast Mistral patching. Transformers: 4.55.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/2.26G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/458 [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
import json
from datasets import Dataset

# Load your JSONL dataset into a list of dicts
with open("finetune_train.jsonl", "r", encoding="utf-8") as f:
    file = [json.loads(line) for line in f]

# No need to reformat — the "text" field already contains User/Assistant + <|endoftext|>
dataset = Dataset.from_dict({"text": [item["text"] for item in file]})

# Sanity check
print(dataset[0])

{'text': 'User: What causes the seasons to change?\nAssistant: The tilt of Earth upon its axis causeth winter’s chill and summer’s warmth, as she doth circle yon blazing sun.<|endoftext|>'}


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r=16,  # Lower rank (saves memory, fine for small dataset)
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha=32,   # 2 × r (balanced)
    lora_dropout=0,  # Optimized for Unsloth
    bias="none",
    use_gradient_checkpointing="unsloth",
    random_state=3407,
    use_rslora=False,
    loftq_config=None,
)


Unsloth 2025.8.9 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,   # effective batch = 8
        warmup_steps=10,
        num_train_epochs=2,              # better for small dataset
        learning_rate=1e-4,              # safer LR
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        logging_steps=10,                # log a bit more often since dataset is tiny
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed=3407,
        output_dir="outputs",
        save_strategy="epoch",
        save_total_limit=2,
        dataloader_pin_memory=False,
        report_to="none",                # disable W&B logging
    ),
)


Unsloth: Tokenizing ["text"]:   0%|          | 0/365 [00:00<?, ? examples/s]

In [ ]:
# Train the model
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 365 | Num Epochs = 2 | Total steps = 92
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 29,884,416 of 3,850,963,968 (0.78% trained)


Step,Training Loss
10,2.967000
20,2.129300
30,1.651800
40,1.486100
50,1.328200
60,1.260400
70,1.247200
80,1.279100
90,1.223100


In [ ]:
import json

# Load held-out test dataset
with open("test_dataset.jsonl", "r") as f:
    test_data = [json.loads(line) for line in f]

print("Test samples:", len(test_data))
print("Example:", test_data[0])


Test samples: 10
Example: {'instruction': 'Summarize the tale of Romeo and Juliet in one sentence.', 'expected': 'Two star-cross’d lovers from feuding houses meet their doom in passion’s final embrace.', 'text': 'User: Summarize the tale of Romeo and Juliet in one sentence.\nAssistant: Two star-cross’d lovers from feuding houses meet their doom in passion’s final embrace.<|endoftext|>'}


In [ ]:
FastLanguageModel.for_inference(model)

results = []
for i, sample in enumerate(test_data[:10]):  # just test 10 first
    messages = [
        {"role": "user", "content": sample["instruction"]}
    ]

    inputs = tokenizer.apply_chat_template(
        messages,
        tokenize=True,
        add_generation_prompt=True,
        return_tensors="pt",
    ).to("cuda")

    outputs = model.generate(
        input_ids=inputs,
        max_new_tokens=256,
        use_cache=True,
        temperature=0.7,
        do_sample=True,
        top_p=0.9,
    )

    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    results.append({
        "instruction": sample["instruction"],
        "expected": sample["expected"],  # <-- changed here
        "predicted": response
    })

# Show results
for r in results:
    print("\nInstruction:", r["instruction"])
    print("Expected:", r["expected"])
    print("Predicted:", r["predicted"])



Instruction: Summarize the tale of Romeo and Juliet in one sentence.
Expected: Two star-cross’d lovers from feuding houses meet their doom in passion’s final embrace.
Predicted: Summarize the tale of Romeo and Juliet in one sentence. Star-crossed lovers wed in passion’s heat, only to meet death’s cold embrace.

Instruction: Explain why the moon doth change her face.
Expected: Because she doth circle Earth, ever shifting her visage as sun’s light falleth upon her sphere.
Predicted: Explain why the moon doth change her face. The moon doth wax and wane, and change her countenance with each passing day, as a mortal woman doth lose and gain her beauty with the turn of time.

Instruction: List two virtues of patience.
Expected: Patience bringeth calm to troubled hearts, and granteth wisdom in the fulness of time.
Predicted: List two virtues of patience. Patience is the virtue that sootheth tempers and maketh the harshest trials bearable.

Instruction: What is the nature of thunder?
Expected

In [ ]:
model.save_pretrained_gguf("gguf_model", tokenizer, quantization_method="q4_k_m")

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 2.3G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 3.9 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 32/32 [00:01<00:00, 20.48it/s]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving gguf_model/pytorch_model-00001-of-00002.bin...
Unsloth: Saving gguf_model/pytorch_model-00002-of-00002.bin...
Done.


Unsloth: Converting mistral model. Can use fast conversion = True.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at gguf_model into f16 GGUF format.
The output location will be /content/gguf_model/unsloth.F16.gguf
This might take 3 minutes...


Unsloth: Extending gguf_model/tokenizer.model with added_tokens.json.
Originally tokenizer.model is of size (32000).
But we need to extend to sentencepiece vocab size (32011).


INFO:hf-to-gguf:Loading model: gguf_model
INFO:hf-to-gguf:Model architecture: MistralForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:gguf: loading model weight map from 'pytorch_model.bin.index.json'
INFO:hf-to-gguf:gguf: loading model part 'pytorch_model-00001-of-00002.bin'
INFO:hf-to-gguf:token_embd.weight,           torch.float16 --> F16, shape = {3072, 32064}
INFO:hf-to-gguf:blk.0.attn_q.weight,         torch.float16 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_k.weight,         torch.float16 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_v.weight,         torch.float16 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.attn_output.weight,    torch.float16 --> F16, shape = {3072, 3072}
INFO:hf-to-gguf:blk.0.ffn_gate.weight,       torch.float16 --> F16, shape = {3072, 8192}
INFO:hf-to-gguf:blk.0.ffn_up.weight,         torch.float16 --> F16, shape = {3072, 8192}
INFO:hf-to-gguf:

In [ ]:
import os

# List all .gguf files
gguf_files = [f for f in os.listdir("gguf_model") if f.endswith(".gguf")]
print("Found GGUF files:", gguf_files)


Found GGUF files: ['unsloth.F16.gguf', 'unsloth.Q4_K_M.gguf']


In [ ]:
import os

for f in ["gguf_model/unsloth.F16.gguf", "gguf_model/unsloth.Q4_K_M.gguf"]:
    if os.path.exists(f):
        print(f, "size (MB):", os.path.getsize(f) / (1024*1024))
    else:
        print(f, "not found")


gguf_model/unsloth.F16.gguf size (MB): 7289.221099853516
gguf_model/unsloth.Q4_K_M.gguf size (MB): 2211.4940490722656


In [ ]:
from google.colab import files
files.download("gguf_model/unsloth.Q4_K_M.gguf")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>